In [3]:
# =========================================================
# 1. Setup
# =========================================================
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import Image, display

# Optional: nicer print formatting in notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# =========================================================
# Board functions
# =========================================================
NUM_ROWS = 6
NUM_COLS = 7

def create_board():
    return np.zeros((NUM_ROWS, NUM_COLS), dtype=int)

def valid_moves(board):
    return np.where(board[0] == 0)[0]

def make_move(board, col, player):
    for r in range(NUM_ROWS - 1, -1, -1):
        if board[r, col] == 0:
            board[r, col] = player
            return

def check_win(board, player):
    # Horizontal
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS - 3):
            if np.all(board[r, c:c+4] == player):
                return True
    # Vertical
    for c in range(NUM_COLS):
        for r in range(NUM_ROWS - 3):
            if np.all(board[r:r+4, c] == player):
                return True
    # Diagonal top-left → bottom-right
    for r in range(NUM_ROWS - 3):
        for c in range(NUM_COLS - 3):
            if np.all([board[r+i, c+i]==player for i in range(4)]):
                return True
    # Diagonal bottom-left → top-right
    for r in range(3, NUM_ROWS):
        for c in range(NUM_COLS - 3):
            if np.all([board[r-i, c+i]==player for i in range(4)]):
                return True
    return False

# =========================================================
# Player functions
# =========================================================
def random_player(board, _):
    return np.random.choice(valid_moves(board))

def heuristic_player(board, player):
    opponent = 3 - player
    moves = valid_moves(board)
    for move in moves:  # try to win
        temp = board.copy()
        make_move(temp, move, player)
        if check_win(temp, player):
            return move
    for move in moves:  # try to block opponent
        temp = board.copy()
        make_move(temp, move, opponent)
        if check_win(temp, opponent):
            return move
    center = NUM_COLS // 2
    if center in moves:
        return center
    return np.random.choice(moves)

def intelligent_player(board, player, depth=1):
    opponent = 3 - player
    best_score = -np.inf
    best_move = valid_moves(board)[0]
    for move in valid_moves(board):
        temp = board.copy()
        make_move(temp, move, player)
        score = minimax(temp, depth-1, False, player, opponent)
        if score > best_score:
            best_score = score
            best_move = move
    return best_move

def minimax(board, depth, maximizing, player, opponent):
    if check_win(board, player): return 100
    if check_win(board, opponent): return -100
    if depth==0 or len(valid_moves(board))==0:
        return evaluate_board(board, player)
    if maximizing:
        return max(minimax(apply_move(board, move, player), depth-1, False, player, opponent)
                   for move in valid_moves(board))
    else:
        return min(minimax(apply_move(board, move, opponent), depth-1, True, player, opponent)
                   for move in valid_moves(board))

def apply_move(board, col, player):
    new_board = board.copy()
    make_move(new_board, col, player)
    return new_board

def evaluate_board(board, player):
    score = 0
    opponent = 3 - player
    for r in range(NUM_ROWS):
        for c in range(NUM_COLS-3):
            window = board[r, c:c+4]
            score += window_score(window, player, opponent)
    return score

def window_score(window, player, opponent):
    score = 0
    if np.count_nonzero(window == player) == 3 and np.count_nonzero(window == 0) == 1:
        score += 10
    if np.count_nonzero(window == opponent) == 3 and np.count_nonzero(window == 0) == 1:
        score -= 8
    return score

def simulate_game(player1_func, player2_func):
    board = create_board()
    current_player = 1
    while True:
        move = player1_func(board, 1) if current_player==1 else player2_func(board, 2)
        make_move(board, move, current_player)
        if check_win(board, current_player):
            return current_player
        if len(valid_moves(board))==0:
            return 0
        current_player = 3 - current_player


# =========================================================
# Monte Carlo
# =========================================================
BASE_SIMULATIONS = 100

players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

starting_advantages = {}

for p1_name, p2_name in combinations(players.keys(), 2):
    p1_wins = 0
    p2_wins = 0
    draws = 0
    for _ in range(BASE_SIMULATIONS):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner==1: p1_wins +=1
        elif winner==2: p2_wins +=1
        else: draws +=1
    advantage = (p1_wins - p2_wins)/BASE_SIMULATIONS
    starting_advantages[f"{p1_name} vs {p2_name}"] = advantage

    # Print results
    print(f"\n{p1_name} vs {p2_name} ({BASE_SIMULATIONS} games):")
    print(f"Player 1 wins: {p1_wins}")
    print(f"Player 2 wins: {p2_wins}")
    print(f"Draws: {draws}")
    print(f"Starting player advantage (P1-P2): {advantage:.3f}")



4 Years vs 7 Years (100 games):
Player 1 wins: 1
Player 2 wins: 99
Draws: 0
Starting player advantage (P1-P2): -0.980

4 Years vs 9 Years (100 games):
Player 1 wins: 15
Player 2 wins: 85
Draws: 0
Starting player advantage (P1-P2): -0.700

4 Years vs 11 Years (100 games):
Player 1 wins: 1
Player 2 wins: 99
Draws: 0
Starting player advantage (P1-P2): -0.980

7 Years vs 9 Years (100 games):
Player 1 wins: 100
Player 2 wins: 0
Draws: 0
Starting player advantage (P1-P2): 1.000

7 Years vs 11 Years (100 games):
Player 1 wins: 41
Player 2 wins: 44
Draws: 15
Starting player advantage (P1-P2): -0.030

9 Years vs 11 Years (100 games):
Player 1 wins: 0
Player 2 wins: 100
Draws: 0
Starting player advantage (P1-P2): -1.000
